In [28]:
import numpy as np
import math
import json
def generateVocab(filename):
    with open('data/'+filename, 'r') as data:
            lines = data.read().splitlines() 
    occurences = {}
    doccurence = {}
    appearances = {}
    uniquewords = {}
    for x in range(len(lines)):
        appearances[x] = {}
        currentLine = lines[x].split(' ')
        uniquewords[x] = list(filter(lambda x: x.isalnum(), list(set(currentLine))))
        alreadyin = {}
        for y in range(1000):
            if currentLine[y].isalnum():
                try:
                    z = alreadyin[currentLine[y]]
                except KeyError:
                    alreadyin[currentLine[y]] = True
                    try:
                        doccurence[currentLine[y]] += 1
                    except KeyError:
                        doccurence[currentLine[y]] = 1
                        
                try:
                    occurences[currentLine[y]] += 1
                except KeyError: 
                    occurences[currentLine[y]] = 1
                finally:
                    try: 
                        appearances[x][currentLine[y]] += 1
                    except KeyError:
                        appearances[x][currentLine[y]] = 1
    occurences = {k: v for k, v in sorted(occurences.items(), key=lambda item: item[1], reverse=True)}
    with open(filename+'unique_words_per_doc.txt', 'w') as fout:
        fout.write(json.dumps(uniquewords))
    with open(filename+'appearances_in_doc.txt', 'w') as fout:
        fout.write(json.dumps(appearances))
    with open(filename+'occurences.txt', 'w') as fout:
        fout.write(json.dumps(occurences))
    with open(filename+'doccurence.txt', 'w') as fout:
        fout.write(json.dumps(doccurence))
    return

In [29]:
generateVocab('training')
generateVocab('testing')

In [30]:
def Bag_of_Word_Feature_Extractor(string):
    occurences = generateVocab()
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    #bag maps word to index
    string = string.split(' ')
    ans = np.zeros(len(occurences.keys()))
    for x in range(len(string)):
        ans[bag[string[x]]] += 1
    return ans
    

In [31]:
def TF_IDF_Feature_Extractor(string):
    with open('occurences.txt', 'r') as fin:
        occurences = json.loads(fin.read())
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    #bag maps word to index
    string = string.split(' ')
    ans = np.zeros(len(occurences.keys()))
    uniqueWords = []
    for z in range(len(string)):
        if string[z] not in uniqueWords:
            uniqueWords.append(string[z])
    with open('data/training', 'r') as data:
        lines = data.read().splitlines() 
    totalDocs = len(lines);
    for x in range(len(uniqueWords)):
        cAppearances = 0
        for y in range(len(string)):
            if string[y] == uniqueWords[x]:
                cAppearances += 1
        #calculate appearances in document
        dApperances = 0;
        for y in range(totalDocs):
            currentLine = lines[y].split(' ')
            if uniqueWords[x] in currentLine:
                dApperances += 1
        #calculate appearances in all documents
        print(uniqueWords[x],  (cAppearances/len(string)) * math.log(totalDocs/dApperances))
        if (dApperances == 0):
            continue
        else:
            ans[bag[uniqueWords[x]]] = (cAppearances/len(string)) * math.log(totalDocs/dApperances)
    return ans
    

In [32]:
def generateFeatureVectors(filename):
    with open('occurences.txt', 'r') as fin:
        occurences = json.loads(fin.read())
    bag = {} 
    counter = 0
    for key in occurences:
        bag[key] = counter
        counter += 1
    result = ()
    kL = len(occurences.keys())
    with open('data/'+filename, 'r') as data:
        lines = data.read().splitlines() 
    with open(filename+'doccurence.txt', 'r') as fin:
        docAp = json.loads(fin.read())
    with open(filename+'unique_words_per_doc.txt', 'r') as fin:
        allUniqueWords = json.loads(fin.read())
    with open(filename+'appearances_in_doc.txt', 'r') as fin:
        cap = json.loads(fin.read())
    totalDocs = len(lines);
    for i, y in enumerate(lines): 
        string = y.split(' ')
        ans = np.zeros(kL)
        uniqueWords = allUniqueWords[str(i)]
        for x in range(len(uniqueWords)):
            cAppearances = cap[str(i)][uniqueWords[x]]
            #calculate appearances in document
            dApperances = docAp[uniqueWords[x]]
            #print(uniqueWords[x],  (cAppearances/len(string)) * math.log(totalDocs/dApperances))
            ans[bag[uniqueWords[x]]] = (cAppearances/(len(string) - 1 )) * math.log(totalDocs/dApperances)
        result = result + (ans,)
    ans = np.row_stack(result)
    np.save(filename+'.npy', ans)
        
        
    

In [33]:
generateFeatureVectors('training')
generateFeatureVectors('testing')

In [34]:
def extractLabels(filename):
    with open('data/'+filename, 'r') as fin:
        data = fin.read().splitlines()
    ans = np.empty(len(data),)
    
    for x in range(len(data)):
        ln = data[x].split(' ')
        ans[x] = int(ln[-1][1:len(ln)+1])
    np.save(filename+'-labels.npy', ans)

In [35]:
extractLabels('training')
extractLabels('testing')